## Sentiment Analysis Experimentation with Starbucks Reviews. Trying to see if I can train a global dataset and apply it to my location, other locations in the district 

In [1]:
%pip install nltk

/Applications/freesurfer/SetUpFreeSurfer.csh: No such file or directory.

[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Importing nltk and ssl so that we can download some packages for a tutorial 
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
])

[nltk_data] Downloading package names to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tonymoceri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to


True

# Compiling Data

In [2]:
# Loading the state of the union corpus we just downloaded. Separating the words. str.isalpha() is to include only the words that are made up of letters
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()]

In [4]:
# Common words such as "a", "the", "of", etc. are referred to as "stopwords". NLTK can essentially remove/ignore these words using a function called .stopwords... 
# Because the corpus contains stopwords in multiple languages, we want to include the "english" argument here
stopwords = nltk.corpus.stopwords.words("english")

# Now, we can remove the stopwords from our words variable and limit the scope of the corpus a bit... 
words = [w for w in words if w.lower() not in stopwords]
words

['PRESIDENT',
 'HARRY',
 'TRUMAN',
 'ADDRESS',
 'JOINT',
 'SESSION',
 'CONGRESS',
 'April',
 'Mr',
 'Speaker',
 'Mr',
 'President',
 'Members',
 'Congress',
 'heavy',
 'heart',
 'stand',
 'friends',
 'colleagues',
 'Congress',
 'United',
 'States',
 'yesterday',
 'laid',
 'rest',
 'mortal',
 'remains',
 'beloved',
 'President',
 'Franklin',
 'Delano',
 'Roosevelt',
 'time',
 'like',
 'words',
 'inadequate',
 'eloquent',
 'tribute',
 'would',
 'reverent',
 'silence',
 'Yet',
 'decisive',
 'hour',
 'world',
 'events',
 'moving',
 'rapidly',
 'silence',
 'might',
 'misunderstood',
 'might',
 'give',
 'comfort',
 'enemies',
 'infinite',
 'wisdom',
 'Almighty',
 'God',
 'seen',
 'fit',
 'take',
 'us',
 'great',
 'man',
 'loved',
 'beloved',
 'humanity',
 'man',
 'could',
 'possibly',
 'fill',
 'tremendous',
 'void',
 'left',
 'passing',
 'noble',
 'soul',
 'words',
 'ease',
 'aching',
 'hearts',
 'untold',
 'millions',
 'every',
 'race',
 'creed',
 'color',
 'world',
 'knows',
 'lost',
 'he

Since asll words in the stopwords list are lowercase, and those in the original list may not be, you can use str.lower() to account for any discrpencies. Otherwise you may end up with mixed case or capitalized stop words still in your list

It is possible to build your own text corpora from any source. Building a corpus can be as simple as loading some plain text or as complex as labeling and categorizing each sentence. We can find NLTK's documentation on how to work with corpus readers. 

For this tutorial, we are just going to use the built-in corpora provided by NLTK. 

In [5]:
# NLTK provides nltk.word_tokenize(), a function that splits raw text into individual words. This will deliver simple word lists really well. 

from pprint import pprint

text = """
For some quick analysis, creating a corpus could be overkill.
If all you need is a word list,
there are simpler ways to achieve that goal."""

pprint(nltk.word_tokenize(text), width=79, compact=True)

['For', 'some', 'quick', 'analysis', ',', 'creating', 'a', 'corpus', 'could',
 'be', 'overkill', '.', 'If', 'all', 'you', 'need', 'is', 'a', 'word', 'list',
 ',', 'there', 'are', 'simpler', 'ways', 'to', 'achieve', 'that', 'goal', '.']


# Creating Frequency Distributions 

A frequency distribution is essentially a table that tells you how many times each word appears within a given text. In NLTK, frequency distributions are a specific object type implementend as a distinct class called "FreqDist". This class provides useful operations for word frequency analysis. 

In [6]:
words: list[str] = nltk.word_tokenize(text)
fd = nltk.FreqDist(words)

# This will create a frequency distribution object similar to a Python dictionary but with added features. 

In [7]:
# After building the object, we can use methods like .most_common() and .tabulate() to start visualizing information: 
print(fd.most_common(3))
print(fd.tabulate(3))

[(',', 2), ('a', 2), ('.', 2)]
, a . 
2 2 2 
None


In [11]:
text = nltk.Text(nltk.corpus.state_union.words())
text.concordance("america", lines=5)

Displaying 5 of 1079 matches:
 would want us to do . That is what America will do . So much blood has already
ay , the entire world is looking to America for enlightened leadership to peace
beyond any shadow of a doubt , that America will continue the fight for freedom
 to make complete victory certain , America will never become a party to any pl
nly in law and in justice . Here in America , we have labored long and hard to 


In [12]:
words: list[str] = nltk.word_tokenize(
    """Beautiful is better than ugly. 
    Explicit is better than implicit.
    Simple is better than complex."""
)
text = nltk.Text(words)
fd = text.vocab()
fd.tabulate(3)

    is better   than 
     3      3      3 


In [13]:
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()]
finder = nltk.collocations.TrigramCollocationFinder.from_words(words)

In [14]:
finder.ngram_fd.most_common(2)
finder.ngram_fd.tabulate(2)

  ('the', 'United', 'States') ('the', 'American', 'people') 
                          294                           185 


In [15]:
from nltk.sentiment import SentimentIntensityAnalyzer 
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")


{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [17]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]


In [18]:
from random import shuffle
def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

shuffle(tweets)
for tweet in tweets[:10]:
    print(">", is_positive(tweet), tweet)

> False #Showbox the only thing that works is that naruto cartoon lets keep fingers crossed this isnt the end for our favourite work companion :(
> False @standuptoUKIP You posted this earlier...it wasn't funny then either. Bit desperate aren't you? #UKIP
> False @_Milli__ unlike me :(
> True @1DCrazen no its okay i didnt mind everyone has their opinions :) anyway we're talking first time
> True @zXw0lfXz Hi Adrian, you should have this parcel on Monday or Tuesday :) -Pris
> True RT @Tommy_Colc: Financial Times come out in support of Tories claiming Miliband is "preoccupied w/ inequality". The man who wrote it http:/…
> False RT @UKIP: '@Nigel_Farage on how the EU has a neo colonial approach to Africa and assuages it's conscience with foreign aid http//t.co/iQtg…
> False RT @LabourEoin: Journalists who prefer to talk about Ed Miliband's stumble rather than actual #BBCqt show are admitting Cameron lost     vi…
> True Come and design your own jute bag today :)
> True @luhanqutie follow b